In [1]:
!pip install rank_bm25

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.2 MB/s eta 0:00:00


In [3]:
from rank_bm25 import BM25Okapi
import numpy as np
from tqdm import tqdm
import os
import pickle
import torch
import glob
import json

from sentence_transformers import SentenceTransformer

In [6]:
def tokenize_doc_to_str(doc: dict) -> str:
    tokenized_doc = []
    for key, value in doc.items():
        tokenized_key = key.lower().replace("_", " ")
        tokenized_doc.append(tokenized_key)
        tokenized_doc.append(':')
        if isinstance(value, str):
            tokenized_doc.extend(value)
        else:
            tokenized_doc.extend(str(value))
    return ' '.join(tokenized_doc)

# Take all json files with names that end '_processed' 
base_path = "/kaggle/input/jokerbot-rag/Kaggle_rag_data"
sbert_embeddings_path = "/kaggle/working/sbert_embeddings.pt"

docs = []
for path in glob.glob(f"{base_path}/*_processed.json"):
    with open(path, 'r') as f:
        docs.extend(json.load(f))

# Initialize SentenceTransformer and ensure it uses GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert = SentenceTransformer('sentence-transformers/all-distilroberta-v1', device=device)

str_docs = [tokenize_doc_to_str(doc) for doc in docs]

sbert_embeddings = sbert.encode(str_docs, show_progress_bar=True, convert_to_tensor=True, batch_size=256)
sbert_embeddings = sbert_embeddings.cpu()  # Move to CPU before saving

torch.save(sbert_embeddings, sbert_embeddings_path)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2938 [00:00<?, ?it/s]

In [6]:
# Define the path and split size
sbert_embeddings = torch.load("/kaggle/working/sbert_embeddings.pt")

output_dir = "/kaggle/working/embeddings_parts"
os.makedirs(output_dir, exist_ok=True)
split_size = 1000  # Number of rows per split
embeddings_size = sbert_embeddings.size(0)
# print(sbert_embeddings[0:5].shape)

# Split and save the tensor
for i in range(0, embeddings_size, split_size):
    end_idx = min(i + split_size, embeddings_size)
    part = sbert_embeddings[i:end_idx].clone()
    torch.save(part, os.path.join(output_dir, f"embeddings_part_{i//split_size}.pt"))

print(f"Saved all parts to {output_dir}")

/tmp/ipykernel_30/1355149148.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sbert_embeddings = torch.load("/kaggle/working/sbert_embeddings.pt")


Saved all parts to /kaggle/working/embeddings_parts


In [7]:
# Load and combine
loaded_parts = []
for file in sorted(os.listdir(output_dir)):
    if file.startswith("embeddings_part_") and file.endswith(".pt"):
        part_path = os.path.join(output_dir, file)
        loaded_parts.append(torch.load(part_path))

reconstructed_embeddings = torch.cat(loaded_parts, dim=0)

# Verify the shape
print("Original shape:", sbert_embeddings.shape)
print("Reconstructed shape:", reconstructed_embeddings.shape)

/tmp/ipykernel_30/672009738.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_parts.append(torch.load(part_path))


Original shape: torch.Size([752114, 768])
Reconstructed shape: torch.Size([752114, 768])


In [4]:
!rm -rf /kaggle/working/embeddings_parts/*

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
!rm -rf /kaggle/working/embedding_parts.zip

In [8]:
!zip -r embedding_parts.zip /kaggle/working/embeddings_parts

  adding: kaggle/working/embeddings_parts/ (stored 0%)
  adding: kaggle/working/embeddings_parts/embeddings_part_271.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_451.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_734.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_387.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_360.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_682.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_481.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_173.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_532.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_578.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_351.pt (deflated 7%)
  adding: kaggle/working/embeddings_parts/embeddings_part_212.pt (deflated 7